In [1]:
%matplotlib inline

### 測試介面

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# 模擬資料（未連資料庫）
data = pd.DataFrame({
    "title": ["Love Song", "Jazz Nights", "Rock Anthem", "Dream Pop", "Hip Hop Vibes"],
    "artist": ["Adele", "Miles Davis", "Queen", "Beach House", "Kendrick Lamar"],
    "genre": ["pop", "jazz", "rock", "dream pop", "hip hop"],
    "release_year": [2015, 1970, 1980, 2012, 2017]
})

# 取得所有 genre 選項
genre_options = ["全部"] + sorted(data["genre"].unique().tolist())

# Widget 定義
keyword_input = widgets.Text(description='關鍵字：')
genre_dropdown = widgets.Dropdown(options=genre_options, description='音樂類型：')
year_slider = widgets.IntRangeSlider(value=[2000, 2023], min=1950, max=2025, step=1, description='年代：')
search_button = widgets.Button(description="搜尋", button_style='primary')
output_area = widgets.Output()

# 搜尋事件
def on_search_clicked(b):
    with output_area:
        clear_output()
        kw = keyword_input.value.lower()
        genre = genre_dropdown.value
        min_y, max_y = year_slider.value

        filtered = data.copy()
        if kw:
            filtered = filtered[
                filtered["title"].str.lower().str.contains(kw) |
                filtered["artist"].str.lower().str.contains(kw)
            ]
        if genre != "全部":
            filtered = filtered[filtered["genre"] == genre]
        filtered = filtered[
            (filtered["release_year"] >= min_y) & (filtered["release_year"] <= max_y)
        ]
        display(filtered)

# 綁定事件
search_button.on_click(on_search_clicked)

# 顯示介面
ui = widgets.VBox([
    keyword_input,
    genre_dropdown,
    year_slider,
    search_button,
    output_area
])
display(ui)

### 連接資料庫

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from sqlalchemy import create_engine, text
import pandas as pd

In [2]:
engine = create_engine("mysql+mysqlconnector://root:@localhost/music_db")

測試有沒有連接成功

In [9]:
def search_songs(keyword):
    query = text("""
        SELECT 
            s.title, 
            s.artist, 
            GROUP_CONCAT(g.name ORDER BY g.name SEPARATOR ', ') AS genres,
            s.emotion
        FROM Songs s
        JOIN Song_Genres sg ON s.song_id = sg.song_id
        JOIN Genres g ON sg.genre_id = g.genre_id
        WHERE s.title LIKE :kw OR s.artist LIKE :kw
        GROUP BY s.song_id, s.title, s.artist, s.emotion
        LIMIT 20
    """)
    with engine.connect() as conn:
        df = pd.read_sql(query, conn, params={"kw": f"%{keyword}%"})
    return df
# 測試
search_songs("love")


,title,artist,genres,emotion
0,Let Love Light the Way,"""Elena Of Avalor"" Cast","britpop, indie rock",joy
1,Lovely Ladies,"""Les Misérables Original London Cast"" Ensemble",classical,joy
2,You Dont Love Me Anymore,"""Weird Al"" Yankovic","comedy, pop, rock",joy
3,I Love Rocky Road,"""Weird Al"" Yankovic","comedy, pop, rock",joy
4,My Babys in Love with Eddie Vedder,"""Weird Al"" Yankovic","comedy, pop, rock",anger
5,If That Isnt Love,"""Weird Al"" Yankovic","comedy, pop, rock",joy
6,My Baby's In Love With Eddie Vedder,"""Weird Al"" Yankovic","comedy, pop, rock",anger
7,...And to Those I Love Thanks for Sticking Around,$uicideboy$,"cloud rap, hip hop, trap",anger
8,In the Grace of Your Love,"&ME,Black Coffee,Keinemusik",hip hop,fear
9,Twelve Days To Christmas,'She Loves Me' 2016 Broadway Company,hip hop,joy


### 搜尋介面輸出

In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from sqlalchemy import create_engine, text
import pandas as pd

# 建立資料庫連線
engine = create_engine("mysql+mysqlconnector://root:@localhost/music_db")

# 從 Genres 表取得下拉選單選項
with engine.connect() as conn:
    genres = pd.read_sql("SELECT name FROM Genres ORDER BY name", conn)
genre_options = ["全部"] + genres['name'].dropna().tolist()

# UI 元件定義
keyword_input = widgets.Text(description='關鍵字：', placeholder='輸入歌名或歌手')
genre_dropdown = widgets.Dropdown(options=genre_options, description='音樂類型：')
year_slider = widgets.IntRangeSlider(value=[2000, 2023], min=1950, max=2025, step=1, description='年代範圍：', layout=widgets.Layout(width='70%'))
search_button = widgets.Button(description='搜尋', button_style='primary')
output_area = widgets.Output()

# 搜尋事件函數
def on_search_clicked(b):
    with output_area:
        clear_output()
        try:
            keyword = keyword_input.value
            genre = genre_dropdown.value
            min_year, max_year = year_slider.value

            # ✅ 檢查空值
            if not keyword.strip():
                print("⚠️ 請輸入關鍵字（歌名或歌手）")
                return

            # SQL 查詢
            sql = """
                SELECT 
                    s.title, 
                    s.artist, 
                    GROUP_CONCAT(g.name ORDER BY g.name SEPARATOR ', ') AS genres,
                    YEAR(s.release_date) AS year,
                    s.emotion
                FROM Songs s
                JOIN Song_Genres sg ON s.song_id = sg.song_id
                JOIN Genres g ON sg.genre_id = g.genre_id
                WHERE (s.title LIKE :kw OR s.artist LIKE :kw)
                  AND (:genre = '全部' OR g.name = :genre)
                  AND (s.release_date IS NOT NULL AND YEAR(s.release_date) BETWEEN :min_year AND :max_year)
                GROUP BY s.song_id, s.title, s.artist, s.release_date, s.emotion
                LIMIT 50
            """

            params = {
                "kw": f"%{keyword}%",
                "genre": genre,
                "min_year": min_year,
                "max_year": max_year
            }

            with engine.connect() as conn:
                result = pd.read_sql(text(sql), conn, params=params)

            if result.empty:
                print("😢 找不到符合條件的歌曲，請嘗試其他關鍵字或條件。")
                return

            # 加上 YouTube 搜尋連結
            result["YouTube"] = result.apply(
                lambda row: f"<a href='https://www.youtube.com/results?search_query={'+'.join(row['title'].split())}+{'+'.join(row['artist'].split())}' target='_blank'>🔗</a>",
                axis=1
            )

            display(HTML(result.to_html(escape=False)))
        
        except Exception as e:
            print("🚫 發生錯誤：", str(e))


# 綁定按鈕
search_button.on_click(on_search_clicked)

# 顯示 UI
ui = widgets.VBox([
    keyword_input,
    genre_dropdown,
    year_slider,
    search_button,
    output_area
])
display(ui)




### 人格分類

從 Songs 抓出特徵資料並做 K-means 分群

In [14]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from sqlalchemy import create_engine, text
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 建立連線
engine = create_engine("mysql+mysqlconnector://root:@localhost/music_db")

# 特徵欄位
features = [
    "energy", "danceability", "positiveness", "speechiness",
    "liveness", "acousticness", "instrumentalness"
]



從每個 cluster 抽出 1 首代表歌

In [15]:
# 從資料庫讀出所有歌曲特徵
with engine.connect() as conn:
    df_all = pd.read_sql(f"""
        SELECT song_id, title, artist, {', '.join(features)}
        FROM Songs
        WHERE release_date IS NOT NULL
    """, conn)

df_all = df_all.dropna(subset=features)

# 特徵標準化
scaler = StandardScaler()
X = scaler.fit_transform(df_all[features])

# K-means 分群
kmeans = KMeans(n_clusters=10, random_state=42)
df_all["cluster"] = kmeans.fit_predict(X)

# 每群抽 1 首
df_recommended = df_all.groupby("cluster").apply(lambda g: g.sample(1, random_state=42)).reset_index(drop=True)

# 加上 YouTube 搜尋連結
df_recommended["YouTube"] = df_recommended.apply(
    lambda row: f"https://www.youtube.com/results?search_query={'+'.join(row['title'].split())}+{'+'.join(row['artist'].split())}",
    axis=1
)


/var/folders/gm/lpq7twgj5n92tmnst4sl52xc0000gn/T/ipykernel_58038/4198017817.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_recommended = df_all.groupby("cluster").apply(lambda g: g.sample(1, random_state=42)).reset_index(drop=True)


顯示介面

In [21]:
def show_recommendation_ui(df_recommended, engine):
    from IPython.display import display, clear_output
    import ipywidgets as widgets
    import pandas as pd

    features = [
        "energy", "danceability", "positiveness", "speechiness",
        "liveness", "acousticness", "instrumentalness"
    ]

    def classify_level(value, thresholds=(40, 70)):
        if value < thresholds[0]:
            return "low"
        elif value < thresholds[1]:
            return "mid"
        else:
            return "high"

    output_result = widgets.Output()

    # ✅ 將 Checkbox 與 YouTube 連結一併顯示
    checkboxes = []
    song_items = []

    for _, row in df_recommended.iterrows():
        cb = widgets.Checkbox(value=False)
        cb.song_id = row['song_id']
        cb.youtube = row['YouTube']
        checkboxes.append(cb)

        link = f"<b>{row['title']} - {row['artist']}</b> <a href='{row['YouTube']}' target='_blank'>🔗</a>"
        html = widgets.HTML(value=link)
        song_items.append(widgets.HBox([cb, html]))

    submit_button = widgets.Button(description="提交喜好", button_style='success')

    def on_submit_clicked(b):
        liked_ids = [cb.song_id for cb in checkboxes if cb.value]
        with output_result:
            clear_output()
            if not liked_ids:
                print("⚠️ 請至少勾選一首你喜歡的歌曲")
                return

            id_list_sql = ", ".join(str(sid) for sid in liked_ids)
            sql = f"""
                SELECT
                    AVG(energy) AS energy,
                    AVG(danceability) AS danceability,
                    AVG(positiveness) AS positiveness,
                    AVG(speechiness) AS speechiness,
                    AVG(liveness) AS liveness,
                    AVG(acousticness) AS acousticness,
                    AVG(instrumentalness) AS instrumentalness
                FROM Songs
                WHERE song_id IN ({id_list_sql})
            """

            with engine.connect() as conn:
                avg_row = pd.read_sql(sql, conn).iloc[0]

            user_levels = {f: classify_level(avg_row[f]) for f in features}

            with engine.connect() as conn:
                df_types = pd.read_sql("SELECT * FROM Personality_Types", conn)

            def score(row):
                return sum(row[f"{f}_level"] == user_levels[f] for f in features)

            df_types["match_score"] = df_types.apply(score, axis=1)
            best = df_types.sort_values("match_score", ascending=False).iloc[0]

            print(f"你可能是：{best['personality_type']}")
            print(f"描述：{best['description']}")

    submit_button.on_click(on_submit_clicked)

    ui_box = widgets.VBox(song_items + [submit_button, output_result])
    display(ui_box)





In [23]:
show_recommendation_ui(df_recommended, engine)